In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Basis-URL definieren
basis_url = "https://deweysearchde.pansoft.de/webdeweysearch/executeSearch.html?query="

# Zahlenlisten generieren
zahlen_einstellig = [str(i) for i in range(10)]
zahlen_zweistellig = [f"{i:02d}" for i in range(100)]
zahlen_dreistellig = [f"{i:03d}" for i in range(1000)]
zahlen = zahlen_einstellig + zahlen_zweistellig + zahlen_dreistellig

# Datenstruktur initialisieren
daten = {}

logging.info("Initialisierung abgeschlossen. Beginne mit dem Abrufen der Daten.")


def bestimme_level(td):
    """Bestimmt die Hierarchieebene anhand des Einzugs im HTML."""
    div = td.find("div")
    if div:
        style = div.get("style", "")
        match = re.search(r"(padding-left|margin-left):\s*([\d\.]+)em", style)
        if match:
            em_wert = float(match.group(2))
            level = int(em_wert / 0.8)
            return level
    return 0


def extract_code(code_zelle):
    code = ""
    # Versuchen, den Code aus einem fettgedruckten <span> zu holen
    bold_span = code_zelle.find(
        "span", style=lambda value: value and "font-weight" in value and "bold" in value
    )
    if bold_span:
        # Inneres graues <span> entfernen
        grey_span = bold_span.find(
            "span", style=lambda value: value and "color" in value and "grey" in value
        )
        if grey_span:
            grey_span.extract()
        code = bold_span.get_text(strip=True)
    else:
        # Falls kein fettgedruckter <span>, versuchen, den Code aus dem <a>-Tag zu holen
        a_tag = code_zelle.find("a")
        if a_tag:
            code = a_tag.get_text(strip=True)
            # Prüfen, ob es einen grauen <span> gibt und den Text hinzufügen
            grey_span = code_zelle.find(
                "span",
                style=lambda value: value and "color" in value and "grey" in value,
            )
            if grey_span:
                code += grey_span.get_text(strip=True)
        else:
            logging.warning(f"Code konnte nicht extrahiert werden: {code_zelle}")
    code = code.replace("\xa0", "").strip()
    code = re.sub(r"\s+", "", code)
    return code


def extract_thema(thema_zelle):
    thema = ""
    div = thema_zelle.find("div")
    if div:
        # Versuchen, das Thema aus einem fettgedruckten <span> zu holen
        bold_span = div.find(
            "span",
            style=lambda value: value and "font-weight" in value and "bold" in value,
        )
        if bold_span:
            thema = bold_span.get_text(strip=True)
        else:
            # Falls kein fettgedruckter <span>, versuchen, das Thema aus dem <a>-Tag zu holen
            a_tag = div.find("a")
            if a_tag:
                thema = a_tag.get_text(strip=True)
            else:
                thema = div.get_text(strip=True)
                logging.warning(f"Thema konnte nicht extrahiert werden: {thema_zelle}")
    else:
        thema = thema_zelle.get_text(strip=True)
    thema = thema.replace("\xa0", "").strip()
    thema = re.sub(r"\s+", " ", thema)
    return thema


for zahl in zahlen:
    url = basis_url + zahl
    logging.info(f"Verarbeite URL: {url}")
    try:
        response = requests.get(url)
        html_inhalt = response.text
        soup = BeautifulSoup(html_inhalt, "html.parser")
        tabelle = soup.find("table", {"id": "scheduleResult"})
        if tabelle is None:
            logging.warning(f"Keine relevanten Daten für Zahl {zahl} gefunden.")
            continue  # Keine relevanten Daten für diese Zahl
        zeilen = tabelle.find_all("tr")
        daten[zahl] = {}
        stack = [daten[zahl]]

        for zeile in zeilen:
            zellen = zeile.find_all("td")
            if len(zellen) >= 2 and zellen[0].get_text(strip=True) != "":
                code_zelle = zellen[0]
                thema_zelle = zellen[1]

                # Code und Thema extrahieren
                code = extract_code(code_zelle)
                thema = extract_thema(thema_zelle)

                if not code:
                    continue  # Wenn kein Code extrahiert werden konnte, überspringen

                # Hierarchieebene bestimmen
                level = bestimme_level(thema_zelle)

                # Stack anpassen
                while len(stack) > level + 1:
                    stack.pop()

                aktuelles_dict = stack[-1]

                # 'sub' initialisieren, falls nicht vorhanden
                if "sub" not in aktuelles_dict:
                    aktuelles_dict["sub"] = {}

                # Neuen Eintrag hinzufügen
                aktuelles_dict["sub"][code] = {"topic": thema}

                # In den Stack einfügen
                stack.append(aktuelles_dict["sub"][code])

                # Logging von Code und Thema
                logging.info(f"Gefunden: Code={code}, Thema={thema}")

        logging.info(f"Request [{zahl}]: Daten erfolgreich verarbeitet.")
        # Kurze Pause, um die Serverlast zu reduzieren
        time.sleep(0.5)
    except Exception as e:
        logging.error(f"Fehler beim Abrufen von {url}: {e}")

logging.info("Datenerfassung abgeschlossen. Beginne mit dem Speichern der Daten.")

# Daten als JSON speichern
try:
    with open("daten.json", "w", encoding="utf-8") as f:
        json.dump(daten, f, ensure_ascii=False, indent=2)
    logging.info("Daten erfolgreich als JSON gespeichert.")
except Exception as e:
    logging.error(f"Fehler beim Speichern der Daten: {e}")

logging.info("Skript-Ausführung beendet.")

In [ ]:
import json


def transform_node(node, current_depth, max_depth):
    result = {}
    # Setze den 'topic' als 'name', falls vorhanden
    if "topic" in node:
        result["name"] = node["topic"]
    else:
        result["name"] = ""
    # Verarbeite die 'sub'-Knoten, falls vorhanden und Tiefe nicht überschritten
    if "sub" in node and current_depth < max_depth:
        result["sub"] = {}
        for key, subnode in node["sub"].items():
            result["sub"][key] = transform_node(subnode, current_depth + 1, max_depth)
    return result


def transform_data(data, max_depth):
    transformed = {}
    for key, value in data.items():
        # Gehe in die 'sub'-Ebene, um die Hauptkategorien ('0' bis '9') zu erreichen
        if "sub" in value:
            for subkey, subvalue in value["sub"].items():
                # Nur Schlüssel von Länge 1 (Hauptkategorien) einbeziehen
                if len(subkey) == 1:
                    transformed[subkey] = transform_node(
                        subvalue, current_depth=1, max_depth=max_depth
                    )
    return transformed


# Laden der Originaldaten aus 'daten.json'
with open("daten.json", "r", encoding="utf-8") as f:
    original_data = json.load(f)

# Transformation der Daten mit maximaler Tiefe von 2 (Hauptkategorie + Unterkategorie)
transformed_data = transform_data(original_data, max_depth=2)

# Speichern der transformierten Daten in 'ddc-basic.json'
with open("ddc-basic.json", "w", encoding="utf-8") as f:
    json.dump(transformed_data, f, ensure_ascii=False, indent=2)

print(
    "Die Daten wurden erfolgreich transformiert und in 'ddc-basic.json' gespeichert."
)
